In [1]:
import keras
import numpy as np 
import pandas as pd 
import os, cv2, random
from keras.models import Sequential
from keras.layers import    Input, Dropout, Flatten, Convolution2D, MaxPooling2D, Dense, Activation

TRAIN_DIR = 'train_equal_re/'
TEST_DIR = 'test_v2_re/'

ROWS = 224
COLS = 224
CHANNELS = 3

train_images = [TRAIN_DIR + i for i in os.listdir(TRAIN_DIR)]
test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]

def read_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_COLOR)
    return cv2.resize(img, (ROWS, COLS), interpolation = cv2.INTER_CUBIC)

    
def prep_data(images):
    count = len(images)
    data = np.ndarray((count, CHANNELS, ROWS, COLS), dtype = np.uint8)
    
    for i, image_file in enumerate(images):
        image = read_image(image_file)
        data[i] = image.T
        
        if i % 1 == 0:
            print ('Processed {} of {}'.format(i, count))
            
    return data

train = prep_data(train_images)
test = prep_data(test_images)

print("Train shape: {}".format(train.shape))
print("Test shape:  {}".format(test.shape))

/Users/dhitithakkar/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Processed 0 of 24
Processed 1 of 24
Processed 2 of 24
Processed 3 of 24
Processed 4 of 24
Processed 5 of 24
Processed 6 of 24
Processed 7 of 24
Processed 8 of 24
Processed 9 of 24
Processed 10 of 24
Processed 11 of 24
Processed 12 of 24
Processed 13 of 24
Processed 14 of 24
Processed 15 of 24
Processed 16 of 24
Processed 17 of 24
Processed 18 of 24
Processed 19 of 24
Processed 20 of 24
Processed 21 of 24
Processed 22 of 24
Processed 23 of 24
Processed 0 of 63
Processed 1 of 63
Processed 2 of 63
Processed 3 of 63
Processed 4 of 63
Processed 5 of 63
Processed 6 of 63
Processed 7 of 63
Processed 8 of 63
Processed 9 of 63
Processed 10 of 63
Processed 11 of 63
Processed 12 of 63
Processed 13 of 63
Processed 14 of 63
Processed 15 of 63
Processed 16 of 63
Processed 17 of 63
Processed 18 of 63
Processed 19 of 63
Processed 20 of 63
Processed 21 of 63
Processed 22 of 63
Processed 23 of 63
Processed 24 of 63
Processed 25 of 63
Processed 26 of 63
Processed 27 of 63
Processed 28 of 63
Processed 29 

In [2]:
labels = []
for i in train_images:
    if 'Agnes' in i:
        labels.append(0)
    elif 'Alexander' in i:
        labels.append(1)
    elif 'Allyson' in i:
        labels.append(2)
    elif 'Amelia' in i:
        labels.append(3)
    elif 'Angelo' in i:
        labels.append(4)
    elif 'Anibal' in i:
        labels.append(5)
    elif 'Anthony' in i:
        labels.append(6)
    elif 'Yekaterina' in i:
        labels.append(7)
    else:
        labels.append(8)
        
train = train.reshape(-1,224,224,3)
test = test.reshape(-1,224,224,3)
X_train = train.astype('float32')
X_test = test.astype('float32')
X_train /= 255
X_test /= 255
Y_train = labels
X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
print(Y_train)
labels_test = []
for i in test_images:
    if 'Agnes' in i:
        labels_test.append(0)
    elif 'Alexander' in i:
        labels_test.append(1)
    elif 'Allyson' in i:
        labels_test.append(2)
    elif 'Amelia' in i:
        labels_test.append(3)
    elif 'Angelo' in i:
        labels_test.append(4)
    elif 'Anibal' in i:
        labels_test.append(5)
    elif 'Anthony' in i:
        labels_test.append(6)
    elif 'Yekaterina' in i:
        labels_test.append(7)
    else:
        labels_test.append(8)
Y_test = labels_test
Y_test = np.array(Y_test)
print(Y_test)
print("X_train.shape " + str(X_train.shape))
print("X_test.shape " + str(X_test.shape))

[3 3 3 7 6 1 5 5 1 6 0 7 6 5 1 7 4 4 4 0 2 2 0 2]
[7 3 3 7 7 3 3 7 3 6 6 3 7 7 3 6 1 2 2 0 6 1 0 2 2 0 1 1 0 2 0 1 1 0 2 2 0
 6 0 4 5 0 6 5 4 5 6 6 5 2 2 5 6 6 0 5 5 0 6 3 3 3 2]
X_train.shape (24, 224, 224, 3)
X_test.shape (63, 224, 224, 3)


In [3]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
model = VGG16(weights='imagenet', include_top=False)

In [4]:
print("Training matrix shape ", X_train.shape) #  m X features
print("Testing matrix shape  ", X_test.shape) #  m X features

Training matrix shape  (24, 224, 224, 3)
Testing matrix shape   (63, 224, 224, 3)


In [5]:
features_x=model.predict(X_train)
print(features_x.shape)
X_train = features_x.reshape(24,25088)
print(X_train.shape)

(24, 7, 7, 512)
(24, 25088)


In [6]:
from resnet_dlib_train import feature_gen 
X_facial = []
for img in sorted(os.listdir(TRAIN_DIR)):
    bounding_box, features = feature_gen(TRAIN_DIR, img)
    X_facial.append(features)
X_facial = np.array(X_facial)
X_train = np.hstack((X_train,X_facial))

In [7]:
features_xtest=model.predict(X_test)
print(features_xtest.shape)
X_test=features_xtest.reshape(63,25088)
print(X_test.shape)

(63, 7, 7, 512)
(63, 25088)


In [8]:
from resnet_dlib_test import feature_gen
X_facial_test = []
for img in sorted(os.listdir(TEST_DIR)):
    bounding_box, features = feature_gen(TEST_DIR, img)
    X_facial_test.append(features)
X_facial_test = np.array(X_facial_test)
X_test = np.hstack((X_test,X_facial_test))

In [9]:
print("Training matrix shape ", X_train.shape)
print("Testing matrix shape  ", X_test.shape)

Training matrix shape  (24, 25216)
Testing matrix shape   (63, 25216)


In [11]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
model=Sequential()
#25216
model.add(Dense(128, input_dim=25216, activation='relu',kernel_initializer='glorot_uniform'))
keras.layers.core.Dropout(0.1, noise_shape=None, seed=None)
model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

model.add(Dense(16,input_dim=128,activation='sigmoid'))
keras.layers.core.Dropout(0.2, noise_shape=None, seed=None)
model.add(keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))

model.add(Dense(9,input_dim=16,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=5)

Epoch 1/5
24/24 [==============================] - 1s 26ms/step - loss: 3.3339 - acc: 0.0833
Epoch 2/5
24/24 [==============================] - 0s 3ms/step - loss: 0.7070 - acc: 0.9167
Epoch 3/5
24/24 [==============================] - 0s 3ms/step - loss: 0.4004 - acc: 1.0000
Epoch 4/5
24/24 [==============================] - 0s 3ms/step - loss: 0.3228 - acc: 1.0000
Epoch 5/5
24/24 [==============================] - 0s 3ms/step - loss: 0.2804 - acc: 1.0000


In [12]:
from sklearn.metrics import log_loss
pred = model.predict(X_test, batch_size=32, verbose=0)
print(pred)
print(model.evaluate(X_test,Y_test))

[[0.10206437 0.08367842 0.06826109 0.05744117 0.04864985 0.27055097
  0.1291678  0.21965992 0.02052635]
 [0.11890335 0.08230159 0.03901517 0.05439773 0.03195954 0.40026587
  0.07464591 0.18598878 0.01252201]
 [0.0501957  0.08702508 0.06957064 0.05043021 0.0450711  0.29341283
  0.24628647 0.1366408  0.02136716]
 [0.09403636 0.0837539  0.05111444 0.0594499  0.03865582 0.36844578
  0.0967902  0.18918252 0.01857108]
 [0.05972714 0.0812078  0.07000476 0.06383776 0.05142839 0.26448262
  0.23562501 0.14959835 0.02408815]
 [0.08010388 0.07745542 0.03694222 0.06064007 0.0360207  0.38417137
  0.12537171 0.1852252  0.01406932]
 [0.06458081 0.06719641 0.04299567 0.06323029 0.04853565 0.32522315
  0.18629543 0.1843202  0.01762242]
 [0.07840116 0.08344755 0.03421691 0.05248134 0.03172443 0.4038517
  0.13072954 0.17167293 0.01347443]
 [0.0367666  0.05430862 0.02715372 0.06098257 0.04410891 0.46153134
  0.17159334 0.12642135 0.01713363]
 [0.05536658 0.0929831  0.03290469 0.04150413 0.02445347 0.499644

In [ ]:
model.summary()